In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM, Input, Dense, Bidirectional, Conv2D, Flatten, Dropout, MaxPooling2D, Lambda, Reshape, concatenate, TimeDistributed
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

import os
from PIL import Image
from collections import OrderedDict
import numpy as np

In [3]:
def getImagesTensors(imgFolder):
    i=0
    imgDict=dict()
    files=os.listdir(imgFolder)
    for file in files:
        if i > 99:
            break
        image = tf.io.read_file(imgFolder+"/"+file)
        image = tf.io.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, (50,50))
        image = tf.cast(image, tf.int32)
        imgDict[file]=image
        i+=1
    return imgDict

In [4]:
def getFinalInputs(d1,d2):
  rez=[]
  for key,val in d1.items():
      rez.append([val,d2[key.split(".")[0]+".mp3"]])
  return rez

In [5]:
trainImagesDictionaryTensors=getImagesTensors('gdrive/MyDrive/spectrograms/train')
testImagesDictionaryTensors=getImagesTensors('gdrive/MyDrive/spectrograms/test')
print(trainImagesDictionaryTensors.keys())
print(testImagesDictionaryTensors.keys())

dict_keys(['common_voice_ro_20789264.jpg', 'common_voice_ro_20789259.jpg', 'common_voice_ro_20789199.jpg', 'common_voice_ro_20789240.jpg', 'common_voice_ro_20789304.jpg', 'common_voice_ro_20789255.jpg', 'common_voice_ro_20789228.jpg', 'common_voice_ro_20789252.jpg', 'common_voice_ro_20789185.jpg', 'common_voice_ro_20789262.jpg', 'common_voice_ro_20789261.jpg', 'common_voice_ro_20789263.jpg', 'common_voice_ro_20789257.jpg', 'common_voice_ro_20789195.jpg', 'common_voice_ro_20789320.jpg', 'common_voice_ro_20789310.jpg', 'common_voice_ro_20789182.jpg', 'common_voice_ro_20789317.jpg', 'common_voice_ro_20789314.jpg', 'common_voice_ro_20789235.jpg', 'common_voice_ro_20789226.jpg', 'common_voice_ro_20789196.jpg', 'common_voice_ro_20789302.jpg', 'common_voice_ro_20789200.jpg', 'common_voice_ro_20789335.jpg', 'common_voice_ro_20789300.jpg', 'common_voice_ro_20789177.jpg', 'common_voice_ro_20789272.jpg', 'common_voice_ro_20789281.jpg', 'common_voice_ro_20789258.jpg', 'common_voice_ro_20789319.jpg

In [6]:
def getSentences(filePath,trainImagesDictionaryTensors):
    data = dict()
    with open(filePath,'r',encoding='utf8') as file:
        for line in file:
            splittedLine = line.split("\t")
            mp3FileName = splittedLine[1]
            if mp3FileName.split(".")[0]+".jpg" in trainImagesDictionaryTensors:
                sentence = splittedLine[2].split(" ")
                data[mp3FileName]=sentence
    return data

In [7]:
trainSentencesDictionary = getSentences('gdrive/MyDrive/cv-corpus-6.1-2020-12-11/ro/train.tsv',trainImagesDictionaryTensors)

In [8]:
trainInputList=getFinalInputs(trainImagesDictionaryTensors,trainSentencesDictionary)

In [9]:
def getInputsOutputs(trainInputList,tokenizer):
    dataString = [value[1] for value in trainInputList]
    tokenizer.fit_on_texts(dataString)
    sequences = tokenizer.texts_to_sequences(dataString)
    vocabularySize = len(tokenizer.word_index) + 1
    sequences=pad_sequences(sequences, maxlen=vocabularySize)
    rez = [[0 for _ in range(vocabularySize)] for _ in range(len(dataString))]
    for i in range(len(sequences)):
      for j in range(len(sequences[i])):
        if sequences[i][j]!=rez[i][j]:
            rez[i][sequences[i][j]-1]=1
    return rez,vocabularySize

In [10]:
tokenizer = Tokenizer(num_words=2000, lower=True, oov_token="<UGACHAKA>")
tokenizedSentences,vocabularySize = getInputsOutputs(trainInputList,tokenizer)
imageHeight,imageWidth,channels=50,50,3

In [11]:
# convolutional layers
pictureInputs = Input(shape=(imageHeight, imageWidth, channels))
pictureInputsRescaled = Rescaling(1. / 255)(pictureInputs)
conv2d = Conv2D(64, 3, padding='same', activation='relu')(pictureInputsRescaled)
conv2d = Conv2D(64, 3, padding='same', activation='relu')(conv2d)
conv2d = Conv2D(64, 3, padding='same', activation='relu')(conv2d)
maxPooling = MaxPooling2D()(conv2d)
dropout = Dropout(0.2)(maxPooling)
conv2d = Conv2D(64, 3, padding='same', activation='relu')(dropout)
conv2d = Conv2D(64, 3, padding='same', activation='relu')(conv2d)
conv2d = Conv2D(64, 3, padding='same', activation='relu')(conv2d)
maxPooling = MaxPooling2D()(conv2d)
dropout = Dropout(0.2)(maxPooling)
conv2d = Conv2D(64, 3, padding='same', activation='relu')(dropout)
conv2d = Conv2D(64, 3, padding='same', activation='relu')(conv2d)
conv2d = Conv2D(64, 3, padding='same', activation='relu')(conv2d)
maxPooling = MaxPooling2D()(conv2d)
convOutput = Dropout(0.2)(maxPooling)

#lstm layers
bilstm = TimeDistributed(Bidirectional(LSTM(64, return_sequences=True)))(convOutput)
bilstm = TimeDistributed(Bidirectional(LSTM(64, return_sequences=True)))(bilstm)
bilstm = TimeDistributed(Bidirectional(LSTM(64)))(bilstm)
flatten = Flatten()(bilstm)
#dense
dense = Dense(128, activation='relu')(flatten)
dense = Dense(64, activation='relu')(dense)
output = Dense(vocabularySize, activation='sigmoid')(dense)

#create model
model = Model(inputs=pictureInputs,outputs=output)

model.summary()

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer='adam', metrics=[tf.keras.metrics.BinaryAccuracy()])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 50, 50, 3)]       0         
_________________________________________________________________
rescaling (Rescaling)        (None, 50, 50, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 50, 50, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 50, 50, 64)        36928     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 25, 25, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 25, 25, 64)        0     

In [12]:
trainInputs=np.array([value[0] for value in trainInputList])
trainOutputs = np.array(tokenizedSentences)
model.fit(trainInputs,trainOutputs, epochs=100)

Epoch 1/100
4/4 [==============================] - 15s 1s/step - loss: 0.6923 - binary_accuracy: 0.6047
Epoch 2/100
4/4 [==============================] - 4s 1s/step - loss: 0.6555 - binary_accuracy: 0.6568
Epoch 3/100
4/4 [==============================] - 4s 1s/step - loss: 0.5250 - binary_accuracy: 0.7441
Epoch 4/100
4/4 [==============================] - 4s 1s/step - loss: 0.3675 - binary_accuracy: 0.8356
Epoch 5/100
4/4 [==============================] - 5s 1s/step - loss: 0.2260 - binary_accuracy: 0.9153
Epoch 6/100
4/4 [==============================] - 4s 974ms/step - loss: 0.1389 - binary_accuracy: 0.9697
Epoch 7/100
4/4 [==============================] - 4s 970ms/step - loss: 0.1047 - binary_accuracy: 0.9825
Epoch 8/100
4/4 [==============================] - 4s 979ms/step - loss: 0.0931 - binary_accuracy: 0.9825
Epoch 9/100
4/4 [==============================] - 4s 1s/step - loss: 0.0867 - binary_accuracy: 0.9831
Epoch 10/100
4/4 [==============================] - 4s 980ms/st

In [13]:
def transformProbabilities(prediction):
    return [1 if value > 0.1 else 0 for value in prediction[0]]

In [14]:
prediction=model.predict(np.array([testImagesDictionaryTensors['common_voice_ro_21398602.jpg']]))
prediction=transformProbabilities(prediction)

In [15]:
def getSentencesFromPrediction(indexesList,tokenizer):
    sentence=""
    for integer in indexesList:
      for word, index in tokenizer.word_index.items():
          if index == integer:
              sentence+= word + " "
              break
    return sentence

In [16]:
indexList = [idx for idx,val in enumerate(prediction) if val == 1]
print(getSentencesFromPrediction(indexList,tokenizer))

<UGACHAKA> de este să acest nu în pe 
